# Imports & setup

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
features = ['Was_home',
 'Team_rating',
 'Opp_rating',
 'Rating_difference',
 'Min_2',
 'Start_2',
 'Gls_2',
 'Sh_2',
 'SoT_2',
 'Ast_2',
 'CrdY_2',
 'CrdR_2',
 'xG_2',
 'xA_2',
 'Team_CS_2',
 'Team_score_2',
 'Opp_score_2',
 'Team_result_2',
 'ICT_index_2',
 'FPL_points_2',
 'Baseline_points_2',
 'Bonus_2',
 'Min_4',
 'Start_4',
 'Gls_4',
 'Sh_4',
 'SoT_4',
 'Ast_4',
 'CrdY_4',
 'CrdR_4',
 'xG_4',
 'xA_4',
 'Team_CS_4',
 'Team_score_4',
 'Opp_score_4',
 'Team_result_4',
 'ICT_index_4',
 'FPL_points_4',
 'Baseline_points_4',
 'Bonus_4',
 'Min_30',
 'Start_30',
 'Gls_30',
 'Sh_30',
 'SoT_30',
 'Ast_30',
 'CrdY_30',
 'CrdR_30',
 'xG_30',
 'xA_30',
 'Team_CS_30',
 'Team_score_30',
 'Opp_score_30',
 'Team_result_30',
 'ICT_index_30',
 'FPL_points_30',
 'Baseline_points_30',
 'Bonus_30',
 'DEF',
 'FWD',
 'MID']

info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "FPL_pos", "Price"]
to_predict = ["FPL_points"]

In [4]:
len(features)

61

# Reading from files

In [5]:
df = pd.read_csv("data/upcoming/upcoming_fixtures.csv")
CURRENT_SEASON = "2023-24"

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (70) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
X_test = df[df["Season"] == CURRENT_SEASON][features]
# .reset_index(drop=True)

In [7]:
df.columns

Index(['Unnamed: 0', 'Ast_2', 'Ast_30', 'Ast_4', 'Baseline_points_2',
       'Baseline_points_30', 'Baseline_points_4', 'Bonus_2', 'Bonus_30',
       'Bonus_4', 'CrdR_2', 'CrdR_30', 'CrdR_4', 'CrdY_2', 'CrdY_30', 'CrdY_4',
       'DEF', 'FPL_points', 'FPL_points_2', 'FPL_points_30', 'FPL_points_4',
       'FPL_pos', 'FWD', 'GW', 'Gls_2', 'Gls_30', 'Gls_4', 'ICT_index_2',
       'ICT_index_30', 'ICT_index_4', 'MID', 'Min_2', 'Min_30', 'Min_4',
       'Name_original', 'Opp_rating', 'Opp_score_2', 'Opp_score_30',
       'Opp_score_4', 'Opponent', 'Rating_difference', 'Season', 'Sh_2',
       'Sh_30', 'Sh_4', 'SoT_2', 'SoT_30', 'SoT_4', 'Start_2', 'Start_30',
       'Start_4', 'Team', 'Team_CS_2', 'Team_CS_30', 'Team_CS_4',
       'Team_rating', 'Team_result_2', 'Team_result_30', 'Team_result_4',
       'Team_score_2', 'Team_score_30', 'Team_score_4', 'Was_home', 'xA_2',
       'xA_30', 'xA_4', 'xG_2', 'xG_30', 'xG_4', 'Finished', 'Date', 'Price'],
      dtype='object')

In [8]:
X_test.columns

Index(['Was_home', 'Team_rating', 'Opp_rating', 'Rating_difference', 'Min_2',
       'Start_2', 'Gls_2', 'Sh_2', 'SoT_2', 'Ast_2', 'CrdY_2', 'CrdR_2',
       'xG_2', 'xA_2', 'Team_CS_2', 'Team_score_2', 'Opp_score_2',
       'Team_result_2', 'ICT_index_2', 'FPL_points_2', 'Baseline_points_2',
       'Bonus_2', 'Min_4', 'Start_4', 'Gls_4', 'Sh_4', 'SoT_4', 'Ast_4',
       'CrdY_4', 'CrdR_4', 'xG_4', 'xA_4', 'Team_CS_4', 'Team_score_4',
       'Opp_score_4', 'Team_result_4', 'ICT_index_4', 'FPL_points_4',
       'Baseline_points_4', 'Bonus_4', 'Min_30', 'Start_30', 'Gls_30', 'Sh_30',
       'SoT_30', 'Ast_30', 'CrdY_30', 'CrdR_30', 'xG_30', 'xA_30',
       'Team_CS_30', 'Team_score_30', 'Opp_score_30', 'Team_result_30',
       'ICT_index_30', 'FPL_points_30', 'Baseline_points_30', 'Bonus_30',
       'DEF', 'FWD', 'MID'],
      dtype='object')

In [9]:
file = open("models/GBR.pkl",'rb')
model = pickle.load(file)

In [10]:
model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

# Predicting

In [11]:
def adjust_for_injuries(df):
    active_players = pd.read_csv("data/misc/active_players.csv")
    injured_players = active_players[active_players["chance_of_playing_this_round"] == 0]["name"].to_list()
    df.loc[df["Name_original"].isin(injured_players),"Pred"] = 0
    return df

In [12]:
def get_predictions(model, df, X, all_remaining=False):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    df_predictions = df[(df["Season"] == CURRENT_SEASON)].reset_index(drop=True)
        
    print(len(predictions), df_predictions.shape)
        
    df_predictions.loc[:, "Pred"] = predictions
    preds = df_predictions[info + ["Pred"]]
    
    preds = adjust_for_injuries(preds).sort_values(by=["Pred"], ascending = False)
    
    return preds

In [13]:
preds = get_predictions(model, df, X_test)

16792 (16792, 72)


C:\users\user\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [14]:
# preds[(preds["FPL_pos"] == "GK") & (preds["GW"] == 2)].head(30)

In [15]:
# next gameweek
preds[preds["GW"] == 2].head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
5025,Erling Haaland,2.0,2023-24,Manchester City,Newcastle Utd,1.0,2080.850586,1887.064819,FWD,140.0,14.592472
8810,Julián Álvarez,2.0,2023-24,Manchester City,Newcastle Utd,1.0,2080.850586,1887.064819,FWD,65.0,11.862788
4532,Eddie Nketiah,2.0,2023-24,Arsenal,Crystal Palace,0.0,1921.462769,1763.994995,FWD,55.0,11.344526
13993,Rodrigo Hernandez,2.0,2023-24,Manchester City,Newcastle Utd,1.0,2080.850586,1887.064819,MID,56.0,11.115772
15164,Taiwo Awoniyi,2.0,2023-24,Nott'ham Forest,Sheffield United,1.0,1671.868774,1637.473755,FWD,65.0,10.146509
7981,Joel Matip,2.0,2023-24,Liverpool,Bournemouth,1.0,1943.976074,1662.536621,DEF,50.0,9.886033
3734,Danny Welbeck,2.0,2023-24,Brighton,Wolves,0.0,1832.364258,1715.631348,FWD,60.0,9.687075
12250,Nathan Aké,2.0,2023-24,Manchester City,Newcastle Utd,1.0,2080.850586,1887.064819,DEF,50.0,9.420984
3204,Cody Gakpo,2.0,2023-24,Liverpool,Bournemouth,1.0,1943.976074,1662.536621,FWD,75.0,9.356344
5100,Ethan Horvath,2.0,2023-24,Nott'ham Forest,Sheffield United,1.0,1671.868774,1637.473755,GK,45.0,8.818118


In [16]:
# all preds
preds.head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
5029,Erling Haaland,6.0,2023-24,Manchester City,Nott'ham Forest,1.0,2080.850586,1671.868774,FWD,140.0,19.171986
5057,Erling Haaland,35.0,2023-24,Manchester City,Nott'ham Forest,0.0,2080.850586,1671.868774,FWD,140.0,18.322220
5044,Erling Haaland,22.0,2023-24,Manchester City,Burnley,1.0,2080.850586,1722.802979,FWD,140.0,18.217823
5027,Erling Haaland,4.0,2023-24,Manchester City,Fulham,1.0,2080.850586,1745.365723,FWD,140.0,18.091955
5034,Erling Haaland,11.0,2023-24,Manchester City,Bournemouth,1.0,2080.850586,1662.536621,FWD,140.0,18.048100
5046,Erling Haaland,24.0,2023-24,Manchester City,Everton,1.0,2080.850586,1699.505859,FWD,140.0,17.846411
5058,Erling Haaland,36.0,2023-24,Manchester City,Wolves,1.0,2080.850586,1715.631348,FWD,140.0,17.846411
5024,Erling Haaland,1.0,2023-24,Manchester City,Burnley,0.0,2077.251953,1726.401611,FWD,140.0,17.368052
5059,Erling Haaland,37.0,2023-24,Manchester City,Fulham,0.0,2080.850586,1745.365723,FWD,140.0,17.242180
5048,Erling Haaland,26.0,2023-24,Manchester City,Bournemouth,0.0,2080.850586,1662.536621,FWD,140.0,17.198326


In [17]:
# points all season
pd.DataFrame(preds.groupby(["Name_original"])["Pred"].sum().sort_values(ascending=False)).head(30)

,Pred
Name_original,
Erling Haaland,601.064453
Rodrigo Hernandez,430.877838
Eddie Nketiah,430.447327
Alexander Isak,426.169952
Julián Álvarez,424.434875
Danny Welbeck,345.203217
Joel Matip,340.645447
Jarrod Bowen,327.071014
Taiwo Awoniyi,316.917145


# Saving to file

In [18]:
preds.to_csv("predictions/preds_next_season.csv", index=False)